# Demo

In [290]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from rdflib import Namespace, RDF, BNode
import os
from pyshacl import validate
from rdflib import Graph, SH
from rdflib.util import guess_format
from tasty.shapes_loader import ShapesLoader

# Simple wrapper around all of the shapes
shrap = te.ShapesWrapper(tc.HAYSTACK, tc.V3_9_10)

shrap.bind()
# shrap.bind_composite()

def print_graph(g):
    print(g.serialize(format='turtle').decode('utf-8'))
    
output_directory = os.path.join(os.path.abspath(''), 'example_data/output')
tasty_main_directory = os.path.join(os.path.abspath(''), '../')    
data_graph_filename = 'examples/example_data/input/demo_data.ttl'
data_graph_equip_filename = 'examples/example_data/input/demo_equip_data.ttl'
shapes_graph_equip_filename = 'examples/example_data/input/demo_equip_shapes.ttl'


In [291]:
user_data = ['point-01']
shapes = []
for attribute in dir(shrap):
    if attribute == 'MinimumDischargeAirFlowSetpointShape':
        shapes.append(attribute)

inputs = []
style = {'description_width': 'initial'}
dropdown = widgets.Dropdown(
    options=user_data,
    description='Point to Validate:',
    disabled=False,
    style=style
)
inputs.append(dropdown)

data = shapes
checkboxes = [widgets.Checkbox(value=False, description=label, style=style) for label in data]
select = widgets.VBox(children=checkboxes)
inputs.append(select)

button = widgets.Button(
    description='Validate',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)
inputs.append(button)

display_results = widgets.Label()
display_missing_points = widgets.Label(style=style)
inputs.append(display_results)
inputs.append(display_missing_points)

def on_button_clicked(b):
    constraints = []
    point = dropdown.value
    for checkbox in checkboxes:
        if checkbox.value == True:
            constraints.append(checkbox.description)
    
    input_namespace_uri = 'urn:sample/'
    NAMESPACE = Namespace(input_namespace_uri)
    shape_name = tc.PH_SHAPES_NREL[constraints[0]]
    target_node = NAMESPACE[point]
            
    data_graph = get_data_graph()
    
    sl = ShapesLoader(tc.HAYSTACK)
    shapes_graph = sl.load_all_shapes()
    shapes_graph.bind('sample', SAMPLE)
    
    shapes_graph.add((shape_name, SH.targetNode, target_node))
    
    result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=h_ont)
    conforms, results_graph, results = result
    missing_points = []
    display_results.value=str(conforms)
    
        # Find the Validation Results
    for subject, predicate, object in results_graph.triples((None, RDF.type, SH.ValidationResult)):
        for node in results_graph.objects(subject=subject, predicate=SH.resultMessage):
            print(node)
            display_missing_points.value="FAIL {}".format(node)
    
           
button.on_click(on_button_clicked)

def get_data_graph():
    n = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, data_graph_filename)
    n.parse(f, format='turtle')
    return n

## Point Selection and Validation (scratch)

In [287]:
widgets.GridBox(inputs, layout=widgets.Layout(height='100px', grid_template_columns="repeat(5, 400px)"))

GridBox(children=(Dropdown(description='Point to Validate:', options=('point-01',), style=DescriptionStyle(des…

In [294]:
user_data = ['My-NREL-VAV']
shapes = ['NREL-VAV-Standard']

inputs = []
style = {'description_width': 'auto'}
dropdown = widgets.Dropdown(
    options=user_data,
    description='Equipment to Validate:',
    disabled=False,
    style=style
)
inputs.append(dropdown)

data = shapes
checkboxes = [widgets.Checkbox(value=False, description=label, style=style) for label in data]
select = widgets.VBox(children=checkboxes)
inputs.append(select)

button = widgets.Button(
    description='Validate',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)
inputs.append(button)

display_results = widgets.Label(style=style)
display_missing_points = widgets.Label(style=style)
inputs.append(display_results)
inputs.append(display_missing_points)

def on_button_clicked(b):
    constraints = []
    equip = dropdown.value
    for checkbox in checkboxes:
        if checkbox.value == True:
            constraints.append(checkbox.description)
    
    input_namespace_uri = 'urn:sample/'
    NAMESPACE = Namespace(input_namespace_uri)
    shape_name = tc.PH_SHAPES_NREL[constraints[0]]
    target_node = NAMESPACE[equip]
            
    data_graph = get_data_graph()
    shapes_graph = get_shapes_graph()
    
    shapes_graph.add((shape_name, SH.targetNode, target_node))
    
    # add Instance Equipment as target node to SHACL Functional Groups Shapes
    for s1, p1, o1 in shapes_graph.triples((shape_name, SH.node, None)):
        shapes_graph.add((o1, SH.targetNode, target_node))
    
    result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=h_ont)
    conforms, results_graph, results = result
    print_graph(results_graph)
    display_results.value=str(conforms)
    missing_points = []

    # Find the Validation Results
    for subject, predicate, object in results_graph.triples((None, RDF.type, SH.ValidationResult)):
        for node in results_graph.objects(subject=subject, predicate=SH.sourceShape):
            if isinstance(node, BNode):
                point = results_graph.value(subject=node, predicate=SH.qualifiedValueShape)
                missing_points.append(point)
    
    if missing_points:
        for point in missing_points:
            for s, p, o in shapes_graph.triples((point,  None, None)):
                if 'class' in str(p):
                    point_class = o.split('#')[1]
                
            missing_p = point.split('#')[1]
        display_missing_points.value="{}: equip is missing a point of class {}".format(str(missing_p), point_class)
        
button.on_click(on_button_clicked)

def get_data_graph():
    n = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, data_graph_equip_filename)
    n.parse(f, format='turtle')
    return n

def get_shapes_graph():
    g = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, shapes_graph_equip_filename)
    g.parse(f, format='turtle')
    return g

## Equip Selection and Validation

In [295]:
widgets.GridBox(inputs, layout=widgets.Layout(height='100px', grid_template_columns="repeat(5, 400px)"))

GridBox(children=(Dropdown(description='Equipment to Validate:', options=('My-NREL-VAV',), style=DescriptionSt…

In [229]:
user_data = ['NREL-Equip-Test']
shapes = ['NREL-VAV-Test-Shape']

inputs = []
style = {'description_width': 'auto'}
dropdown = widgets.Dropdown(
    options=user_data,
    description='System to Validate:',
    disabled=False,
    style=style
)
inputs.append(dropdown)

data = shapes
checkboxes = [widgets.Checkbox(value=False, description=label, style=style) for label in data]
select = widgets.VBox(children=checkboxes)
inputs.append(select)

button = widgets.Button(
    description='Validate',
    disabled=False,
    button_style='',
    tooltip='Click me',
    icon='check'
)
inputs.append(button)

display_results = widgets.Label(style=style)
display_missing_points = widgets.Label(style=style)
inputs.append(display_results)
inputs.append(display_missing_points)

def on_button_clicked(b):
    constraints = []
    equip = dropdown.value
    for checkbox in checkboxes:
        if checkbox.value == True:
            constraints.append(checkbox.description)
    
    input_namespace_uri = 'urn:sample/'
    NAMESPACE = Namespace(input_namespace_uri)
    shape_name = tc.PH_SHAPES_NREL[constraints[0]]
    target_node = NAMESPACE[equip]
            
    data_graph = get_data_graph()
    shapes_graph = get_shapes_graph()
    
    shapes_graph.add((shape_name, SH.targetNode, target_node))
    
    # add Instance Equipment as target node to SHACL Functional Groups Shapes
    for s1, p1, o1 in shapes_graph.triples((shape_name, SH.node, None)):
        shapes_graph.add((o1, SH.targetNode, target_node))
    
    result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=h_ont)
    conforms, results_graph, results = result
    display_results.value=str(conforms)
    missing_points = []

    # Find the Validation Results
    for subject, predicate, object in results_graph.triples((None, RDF.type, SH.ValidationResult)):
        for node in results_graph.objects(subject=subject, predicate=SH.sourceShape):
            if isinstance(node, BNode):
                point = results_graph.value(subject=node, predicate=SH.qualifiedValueShape)
                missing_points.append(point)
    
    if missing_points:
        for point in missing_points:
            for s, p, o in shapes_graph.triples((point,  None, None)):
                if 'class' in str(p):
                    point_class = o.split('#')[1]
                
            missing_p = point.split('#')[1]
        display_missing_points.value="FAIL {}: equip is missing a point of class {}".format(str(missing_p), point_class)
        
button.on_click(on_button_clicked)

def get_data_graph():
    n = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, data_graph_equip_filename)
    n.parse(f, format='turtle')
    return n

def get_shapes_graph():
    g = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, shapes_graph_equip_filename)
    g.parse(f, format='turtle')
    return g

## System Selection and Validation

In [230]:
widgets.GridBox(inputs, layout=widgets.Layout(height='100px', grid_template_columns="repeat(5, 400px)"))

GridBox(children=(Dropdown(description='System to Validate:', options=('NREL-Equip-Test',), style=DescriptionS…